In [53]:
import pandas as pd
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch
from torch import Tensor

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base", pad_token="<pad>", sep_token = "<sep>", cls_token= "<CLS>")

model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [54]:
train_data = pd.read_csv("../../data/train/train.csv")
test_data = pd.read_csv("../../data/test/final_test_pairs.csv")

print(train_data.columns)

Index(['Unnamed: 0', 'pair_id', 'id1', 'id2', 'text1', 'text2', 'overall',
       'lang1', 'lang2'],
      dtype='object')


In [55]:
def tokenize_and_shorten_sentence(text: str) -> Tensor:
    """
    Tokenize the input text and shorten it to 256 tokens.

    Args:
        text (str): The input text.

    Returns:
        Tensor: The tokenized and shortened text tensor.
    """

    tokenized_text = tokenizer(text, return_tensors="pt", padding=False, truncation=False) #padding = False and truncation = False to get the exact length of the text
    if tokenized_text["input_ids"].shape[1] > 256:
        tokenized_text["input_ids"] = torch.cat((tokenized_text["input_ids"][:, :200], tokenized_text["input_ids"][:, -56:]), dim=1)
        tokenized_text["attention_mask"] = torch.cat((tokenized_text["attention_mask"][:, :200], tokenized_text["attention_mask"][:, -56:]), dim=1)
    else:
        tokenized_text["input_ids"] = tokenizer(text, return_tensors="pt", padding="max_length", truncation=False, max_length=256)["input_ids"]
        tokenized_text["attention_mask"] = tokenizer(text, return_tensors="pt", padding="max_length", truncation=False, max_length=256)["attention_mask"]
    
    return tokenized_text

In [81]:
input_ids = torch.empty((len(train_data), 512), dtype=torch.long)
attention_mask = torch.empty((len(train_data), 512), dtype=torch.long)

for i in range(len(train_data)):
    text1 = train_data["text1"][i]
    text2 = train_data["text2"][i]
    score = train_data["overall"][i]

    text1_tokenized = tokenize_and_shorten_sentence(text1)
    text2_tokenized = tokenize_and_shorten_sentence(text2)

    input_ids[i] = torch.cat((text1_tokenized["input_ids"], text2_tokenized["input_ids"]), dim=1)
    attention_mask[i] = torch.cat((text1_tokenized["attention_mask"], text2_tokenized["attention_mask"]), dim=1)



In [86]:
model_output = model(input_ids=input_ids, attention_mask=attention_mask)

IndexError: index out of range in self

In [70]:
input_ids.shape

torch.Size([4875, 512])